In [3]:
import requests
import csv 
from collections import Counter
from pandas import *

In [4]:
listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    s=requests.session()  #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',} #send the login data
    s.post("https://www.icheckmovies.com/login/", data=login_data) #csv link 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(toplistnum) is True:
            toplistnum=int(row[5])
        toplist.append(toplistnum)        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

def process_club_list(user_list,others):
    merge_frames = others.merge(user_list,on=['title','most_checked'],how='left')
    not_same_titles = merge_frames[merge_frames.top_lists_y.isnull()]
    del not_same_titles['top_lists_y']
    not_same_titles.columns=['most_checked', 'title', 'top_lists']
    return(not_same_titles)

#this is working on a function that will do everything
def Listmaker_pro(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    user1_list=listmaker(username1)
    user2_list=listmaker(username2)
    user3_list=listmaker(username3)
    combo_1_2=user1_list.append(user2_list)
    seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]
    combo_1_3=user1_list.append(user3_list)
    seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]
    combo_2_3=user2_list.append(user3_list)
    seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
    club1_list = process_club_list(user1_list,seenby_2_3) 
    club2_list = process_club_list(user2_list,seenby_1_3)
    club3_list = process_club_list(user3_list,seenby_1_2)

In [5]:
username1='Haessar'
username2='hermesmovies'
username3='James88'
password='widmerpool88'

In [6]:
csvgetter(username1,password,username2,username3)

There are 3 users in the database: 
hermesmovies_checklist.csv
James88_checklist.csv
Haessar_checklist.csv


In [7]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user3 is "+username3+", user#_list is a dataframe of each users list")

user1 is Haessar, user2 is hermesmovies and user3 is James88, user#_list is a dataframe of each users list


Want to make a list that both james and will have seen that I haven't seen

In [8]:
#join User 1 and User 2
combo_1_2=user1_list.append(user2_list)
#return duplicates
seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]

#join User 1 and User 3
combo_1_3=user1_list.append(user3_list)
seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]

#join User 2 and User 3
combo_2_3=user2_list.append(user3_list)
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 



In [9]:
#Club list for User 1:
club1_list = process_club_list(user1_list,seenby_2_3) 

#Club list for User 2:
club2_list = process_club_list(user2_list,seenby_1_3)

#Club list for User 3:
club3_list = process_club_list(user3_list,seenby_1_2)


IDEA FOR FINAL LIST. MAKE TWO LISTS, ONE LISTED BY CHECKED AND ONE BY TOPLISTS THEN LOOP APPEND THEM ALTERNATLY TO A NEW LIST, CHECKING EACH LOOP THAT IT IS NOT IN THE LIST ALREADY


In [10]:
by_checked=club2_list.sort(['most_checked'],ascending=False)
by_toplists=club2_list.sort(['top_lists'],ascending=False)

finaldata=DataFrame(columns=('most_checked','title','top_lists'))
listof=Series(club2_list.iloc[1])
newframe=finaldata.append(listof,ignore_index=True)
print(newframe)
listof=Series(club2_list.iloc[4])
newframe=newframe.append(listof,ignore_index=True)
print(newframe)
listof=Series(club2_list.iloc[6])
newframe=newframe.append(listof,ignore_index=True)
print(newframe)

print(newframe.iloc[[1,1]])
len(newframe.loc[newframe['title'] == 'Don Jon'])

print(listof[1])

def isinlist(fromlistof):
    if (len(newframe.loc[newframe['title'] == listof[1]])) is 1:
        return True
    else:
        return False


   most_checked    title  top_lists
0         20381  Traffic          6
   most_checked        title  top_lists
0         20381      Traffic          6
1          4447  La pianiste          5
   most_checked                        title  top_lists
0         20381                      Traffic          6
1          4447                  La pianiste          5
2          5736  The Texas Chainsaw Massacre          0
   most_checked        title  top_lists
1          4447  La pianiste          5
1          4447  La pianiste          5
The Texas Chainsaw Massacre


In [49]:
counter=3
rec=by_checked.iloc[[(counter-1)/2]]
print(rec)

     most_checked          title  top_lists
277         40273  Sleepy Hollow          3


In [67]:
def iseven(number):
    if number %2==0:
        return True
    else:
        return False

def lookupinlist(newtitle):
    if newframe.loc[newframe['title'] == newtitle]:
        return True
    else:
        return False

def final_table(username):
    by_checked=club1_list.sort(['most_checked'],ascending=False)
    by_toplists=club1_list.sort(['top_lists'],ascending=False)
    finaldata=DataFrame(columns=('most_checked','title','top_lists'))
    for counter in range(0, 10):
        #if isinlist(listof[counter]) is False:
        if iseven(counter) is True:
            rec = by_checked.iloc[[counter/2]]
            finaldata=finaldata.append(rec,ignore_index=True)
            #print(rec)
        else:
            rec = by_toplists.iloc[[(counter-1)/2]]
            finaldata=finaldata.append(rec,ignore_index=True)
            #print(rec)
    print(finaldata)
    

In [ ]:
club2_list.sort(['top_lists','most_checked'],ascending=False)

In [68]:
final_table("Haessar")

   most_checked                title  top_lists
0         56374                 Léon         18
1         56374                 Léon         18
2         52834           Zombieland          2
3         31617             Magnolia         16
4         51906  Night at the Museum          2
5         50557             Rain Man         14
6         50557             Rain Man         14
7         50518       Twelve Monkeys         14
8         50518       Twelve Monkeys         14
9         30144              RoboCop         12
